# 6.1 - Macrobond Data API - Update Your Universe

*Retreive your universe, check and synchronise your database for updates*

This is a notebook example that demonstrate some of the features of the SubscriptionList endpoints. For comprehensive usage example, please refer to example `subscribing_to_updates.py`

Please note that SubscriptionList endpoints would only work with real PrimNames, not with the aliases. For example, you should set "usnaac0169" instead of "usgdp" for the SubscriptionList. You can find PrimNames in each entities metadata.

Besides, apart from TimeSeries we demonstrate below, you can also use SubscriptionList endpoints to check updates of other entities like Release, Source, etc.

*Full error handling is omitted for brevity*

***

## Importing packages

In [0]:
from macrobond_data_api.web import WebClient

api = WebClient().open()

from datetime import datetime, timedelta, timezone

***

## Define the universe for checking the update


For this example only, we are using a Search query to build a master list of time series.
But, in practice, this should be the universe you wish to subscribe to on a production basis, and you want to keep them up to date.

In [ ]:
data_frame = api.entity_search(
    entity_types="TimeSeries",
    must_have_values={"Category": "pric", "Frequency": "daily", "Region": "cn"},
).to_pd_data_frame()
data_frame.shape

Now, let's select and download the first 100 time series from the query above.

In [0]:
# example using get_many_series
myseries = list(data_frame["Name"])[
    :100
]  # select the first 100 series of the result above as an example
database = {}
for i in api.get_many_series(
    [(i) for i in myseries]
):  # extract the value from Macrobond
    database[i.primary_name] = i.to_dict()  # transfer the value into your database

***

## Set up the Subscription List, poll for updates, and retrieve updates time series

Here, we set the Subscription List as our master universe, retrieved from the section just above. We also set up the last update time/download time of the universe in our database.

In practice, please make sure your input universe is effective entities PrimName, instead of aliases or anything else.

For demonstration purpose only, we want to see some update information, so let's set the last update time to 3 days ago. In a real case, you should set it to the time you downloaded the universe last.

We also set the poll_inverval to 20 seconds. This means the function will pause 20 seconds once it finishes checking every time series in the universe, before it starts checking again.

In [0]:
list_modified = datetime.now(timezone.utc) - timedelta(
    days=3
)  # Let us assume the last time you update your universe is three days ago, so we can get some update info below

subscription_list = api.subscription_list(
    list_modified, poll_interval=timedelta(seconds=20)
)

subscription_list.set(myseries)

### 1. Continuous updates - `poll` function

You can choose using the poll function to keep checking the update information. If it is not interrupted, it acts as a pulse, keeping your database synchronised with Macrobond.

The example below is designed for illustration purpose only where the polling frequency is set to a fixed interval.
In a production environment, where you want to consistently keep checking for updates, this would be achieved through an infinite loop with a statement such as 'while true'.

In [ ]:
time = datetime.now(timezone.utc)
twenty_seconds_later = time + timedelta(seconds=20)

while (
    time <= twenty_seconds_later
):  # in your environment this should be a infinite while loop (while True), here we set a 20 seconds count down for illustration purpose.
    result = subscription_list.poll()  # poll and check update information constantly
    print(result)
    if (
        result != {}
    ):  # once there are any update detected here, put the updated timeseries in get_many_series or get_series and sync the value to your database.
        updated_timeseries_extract = api.get_series(list(result.keys()))
        for i in range(len(updated_timeseries_extract)):
            database[updated_timeseries_extract[i].primary_name] = (
                updated_timeseries_extract[i].to_dict()
            )
    time = datetime.now(timezone.utc)

### 2. One-off updates - `poll_until_no_more_changes` function

Or you can choose to use `poll_until_no_more_changes` function, this will poll update information in one go. It will stop polling once no further updates are detected, rather than pausing and restarting.

This method is more suited for environments that aren't expected to run continuously with an infinite loop.

In [ ]:
subscription_list = api.subscription_list(
    list_modified, poll_interval=timedelta(seconds=20)
)
subscription_list.set(myseries)


result = subscription_list.poll_until_no_more_changes()
universe_to_be_updated = sum([list(i.keys()) for i in result], [])
print(universe_to_be_updated)
if (
    result != {}
):  # once there are any update detected here, put the updated timeseries in get_many_series or get_series and sync the value to your database.
    updated_timeseries_extract = api.get_series(universe_to_be_updated)
    for i in range(len(updated_timeseries_extract)):
        database[updated_timeseries_extract[i].primary_name] = (
            updated_timeseries_extract[i].to_dict()
        )

***

## Check universe in the Subscription List

In [ ]:
subscription_list.list()

***

## Delete an entity in the subscription list

Note: You should not make several calls in parallel that modifies the list

In [0]:
subscription_list.remove(
    subscription_list.list()
)  # remove all the series in the subscription list

***

## Add entities to the subscription list

In [0]:
subscription_list.add(myseries)  # add a list of entities